In [22]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
plt.style.use ('ggplot')
import seaborn as sns
%matplotlib inline
sns.set_style(style="whitegrid")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

from bs4 import BeautifulSoup

import nltk, string, contractions, random
from nltk.stem.snowball import SnowballStemmer

from gensim.corpora.dictionary import Dictionary
from gensim.models.nmf import Nmf
from gensim.models.coherencemodel import CoherenceModel

from operator import itemgetter

In [99]:
df1 = pd.read_csv('data/modeling_ready_microwave1')
df2 = pd.read_csv('data/modeling_ready_microwave2')
df3 = pd.read_csv('data/modeling_ready_microwave3')

In [100]:
df1.dropna(inplace=True)
df2.dropna(inplace=True)
df3.dropna(inplace=True)

In [101]:
df1.review_date = pd.to_datetime(df1.review_date)
df2.review_date = pd.to_datetime(df2.review_date)
df3.review_date = pd.to_datetime(df3.review_date)

In [26]:
keep_words = ['not', 'ain', 'aren', "aren't", 'couldn', "couldn't", 
              'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 
              'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 
              'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn',
              "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 
              'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 
              'wouldn', "wouldn't", 'fire', 'off']
test_stop_words = []
review_stop_words = ['panasonic', 'really', 'husband', 'thanks', 'thank', 'ha', 
              'just', 'thing', 'did', 'nn', 'wa', 'yr', 'u', 'say', 'doe',
              'mom', 'christmas', 'gift', 'got', 'way', 'le', 'daughter',
              'e','not','love','good','bought','great microwave','micro',
              'great oven','microwave','product','work great','nice work',
              'work great use','work great love','feature work great',
              'unit work great','oven work great','easy use love','old oven',
              'old old','20 year old','unit', 'not', 'work', 'amazon', 'com',
              'old', 'wife', 'highly', 'recommend', 'like', 'charm', '20', 
              'easy', 'oven', 'use', 'year', 'lot', 'pleased', 'happy', 'hope',
              'review', 'buy', 'far', 'day', '1', '2', '3', '4', '5', '6', '7',
              '8', '9', '10', '20', 'great', 'excellent','feature', 'nice', 'old']
for _ in text.ENGLISH_STOP_WORDS:
    if _ in keep_words:
        pass
    else:
        review_stop_words.append(_)
for _ in test_stop_words:
    review_stop_words.append(_)   
    
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()    
    
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

def vectorize_this(df, max_features, min_df, max_df, ngram_max):
    vectorizer = TfidfVectorizer(tokenizer = lemmatize_text,
                             stop_words= review_stop_words,
                             max_features = max_features,
                             min_df = min_df,
                             max_df =  max_df,
                             ngram_range=(1, ngram_max)
                            )
    X = df['model_ready']
    X = vectorizer.fit_transform(X)
    return X, vectorizer

In [27]:
topics1 = 7
mindf1 = 2
maxdf1 = .99
grams1 = 3

In [ ]:
# topics2 = 7
# mindf2 = 2
# maxdf2 = .99
# grams2 = 3

In [ ]:
# topics3 = 7
# mindf3 = 2
# maxdf3 = .99
# grams3 = 3

In [28]:
X1, vectorizer1 = vectorize_this(df1, 10000, mindf, maxdf, grams)

In [ ]:
# X2, vectorizer2 = vectorize_this(df2, 10000, mindf, maxdf, grams)

In [ ]:
# X3, vectorizer3 = vectorize_this(df3, 10000, mindf, maxdf, grams)

In [29]:
def make_nmf(n_components, alpha, X):
    nmf = NMF(
            n_components=n_components,
            init='nndsvd',
            random_state=12345,
            alpha = alpha
            ).fit(X)

    W = nmf.fit_transform(X)
    H = nmf.components_
    return nmf, W, H

In [227]:
nmf1, W1, H1, = make_nmf(topics, .001, X1)

In [ ]:
# nmf2, W2, H2, = make_nmf(topics, .1, X2)

In [ ]:
# nmf3, W3, H3, = make_nmf(topics, .1, X3)

In [233]:
def topic_keywords(nmf_model, vectorizer=vectorizer, n_words=10):

    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in nmf_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [236]:
np.array(vectorizer1.get_feature_names()).take(nmf1.components_[1].argsort()[:10])

array(['job month', 'market', 'transaction', 'transferred',
       "manufacturer's", 'manufacturer warranty',
       'manufacturer produce sell', 'manufacturer produce',
       'manufacturer build', 'market new'], dtype='<U31')

1280

In [211]:
df1.review_body[1280]

'Shipped like the speed of light! Works great, very attractive in white, too. Yep, pretty good deal, too. Okey dokey.'

In [237]:
topic_keywords1

[array(['time', 'popcorn', 'button', 'cooking', 'setting', 'minute', 'bag',
        'used', 'second', 'long'], dtype='<U31'),
 array(['door', 'open', 'latch', 'close', 'door latch', 'push', 'button',
        'door open', 'problem', 'hard'], dtype='<U31'),
 array(['lasted', 'month', 'working', 'warranty', 'service', 'repair',
        'died', 'new', 'model', 'purchased'], dtype='<U31'),
 array(['price', 'large', 'fit', 'size', 'big', 'space', 'counter', 'need',
        'kitchen', 'needed'], dtype='<U31'),
 array(['power', 'level', 'power level', 'watt', 'inverter', '1250',
        '1250 watt', 'setting', 'power setting', 'watt power'],
       dtype='<U31'),
 array(['sensor', 'reheat', 'sensor reheat', 'food', 'defrost', 'function',
        'button', 'reheat function', 'sensor cook', 'used'], dtype='<U31'),
 array(['cook', 'food', 'heat', 'evenly', 'fast', 'quickly', 'cook food',
        'cook fast', 'heat food', 'powerful'], dtype='<U31')]

In [120]:
W1.shape, H1.shape

((1848, 7), (7, 8511))

In [117]:
nmf1.components_.shape

(7, 8511)

In [235]:
topic_keywords1 = topic_keywords(nmf1, vectorizer1)        

In [ ]:
# topic_keywords2 = topic_keywords(vectorizer2, nmf2)        

In [ ]:
# topic_keywords3 = topic_keywords(vectorizer3, nmf3)        

In [35]:
topic_keywords1

[array(['time', 'popcorn', 'button', 'cooking', 'setting', 'minute', 'bag',
        'used', 'second', 'long'], dtype='<U31'),
 array(['door', 'open', 'latch', 'close', 'door latch', 'push', 'button',
        'door open', 'problem', 'hard'], dtype='<U31'),
 array(['lasted', 'month', 'working', 'warranty', 'service', 'repair',
        'died', 'new', 'model', 'purchased'], dtype='<U31'),
 array(['price', 'large', 'fit', 'size', 'big', 'space', 'counter', 'need',
        'kitchen', 'needed'], dtype='<U31'),
 array(['power', 'level', 'power level', 'watt', 'inverter', '1250',
        '1250 watt', 'setting', 'power setting', 'watt power'],
       dtype='<U31'),
 array(['sensor', 'reheat', 'sensor reheat', 'food', 'defrost', 'function',
        'button', 'reheat function', 'sensor cook', 'used'], dtype='<U31'),
 array(['cook', 'food', 'heat', 'evenly', 'fast', 'quickly', 'cook food',
        'cook fast', 'heat food', 'powerful'], dtype='<U31')]

In [36]:
# def topic_featuring(X, topic_keywords, n_components=10, n_words=10):
    
#     nmf, W, H = make_nmf(n_components, .1, X)
    
#     # Topic - Keywords Dataframe
#     df_topic_keywords = pd.DataFrame(topic_keywords)
#     df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
#     df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

#     Topics_theme = range(n_components)
#     df_topic_keywords['topic_theme'] = Topics_theme
#     df_topic_keywords.set_index('topic_theme', inplace=True)
#     return df_topic_keywords.T

In [37]:
# topic_featuring(X1, topic_keywords1, topics1)

topic_theme,0,1,2,3,4,5,6
Word 0,time,door,lasted,price,power,sensor,cook
Word 1,popcorn,open,month,large,level,reheat,food
Word 2,button,latch,working,fit,power level,sensor reheat,heat
Word 3,cooking,close,warranty,size,watt,food,evenly
Word 4,setting,door latch,service,big,inverter,defrost,fast
Word 5,minute,push,repair,space,1250,function,quickly
Word 6,bag,button,died,counter,1250 watt,button,cook food
Word 7,used,door open,new,need,setting,reheat function,cook fast
Word 8,second,problem,model,kitchen,power setting,sensor cook,heat food
Word 9,long,hard,purchased,needed,watt power,used,powerful


In [ ]:
# topic_featuring(X2, topic_keywords2, topics2)

In [ ]:
# topic_featuring(X3, topic_keywords3, topics3)

In [55]:
def df_featurizer(df, topic_keywords, topic_labels=None, n_components=10, column_names_known = 'n'):

    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
    
    if column_names_known == 'n':
        Topics_theme = range(n_components)
    elif column_names_known == 'Y':
        Topics_theme = topic_labels
    df_topic_keywords['topic_theme'] = Topics_theme
    df_topic_keywords.set_index('topic_theme', inplace=True)
    return df_topic_keywords.T

In [56]:
df_featurizer(df1, topic_keywords1, n_components=topics1)

topic_theme,0,1,2,3,4,5,6
Word 0,time,door,lasted,price,power,sensor,cook
Word 1,popcorn,open,month,large,level,reheat,food
Word 2,button,latch,working,fit,power level,sensor reheat,heat
Word 3,cooking,close,warranty,size,watt,food,evenly
Word 4,setting,door latch,service,big,inverter,defrost,fast
Word 5,minute,push,repair,space,1250,function,quickly
Word 6,bag,button,died,counter,1250 watt,button,cook food
Word 7,used,door open,new,need,setting,reheat function,cook fast
Word 8,second,problem,model,kitchen,power setting,sensor cook,heat food
Word 9,long,hard,purchased,needed,watt power,used,powerful


In [ ]:
# nmf_featurizer(df2, max_df=maxdf2, ngram_max=grams2, n_components=topics2)

In [ ]:
# nmf_featurizer(df3, max_df=maxdf3, ngram_max=grams3, n_components=topics3)

In [238]:
vocabulary1 = np.array(vectorizer1.get_feature_names())
vocabulary1

array(["'11", "'enjoy", "'enjoy meal'", ..., 'zapped', 'zero',
       'zero star'], dtype='<U31')

In [ ]:
# vocabulary2 = np.array(vectorizer2.get_feature_names())

In [ ]:
# vocabulary3 = np.array(vectorizer3.get_feature_names())

In [46]:
def label_topics(H, vocabulary):
    '''
    Print the most influential words of each latent topic, and prompt the user
    to label each topic. The user should use their humanness to figure out what
    each latent topic is capturing.
    '''
    topic_labels = []
    for i, row in enumerate(H):
        top_five = np.argsort(row)[::-1][:12]
        print('topic', i)
        print('-->', ' '.join(vocabulary[top_five]))
        label = input('please label this topic: ')
        topic_labels.append(label)
        print()
    return topic_labels

In [239]:
topic_labels1 = label_topics(H1,vocabulary1)

topic 0
--> time popcorn button cooking setting minute bag used second long pop popcorn setting


KeyboardInterrupt: 

In [ ]:
# topic_labels2 = label_topics(H2,vocabulary2)

In [ ]:
# topic_labels3 = label_topics(H3,vocabulary3)

In [63]:
df_featurizer(df1, topic_keywords1, topic_labels1, n_components=topics1, column_names_known = 'Y')

topic_theme,popcorn,door,short lifespan,size,power,programmed functions,cooks fast and evenly
Word 0,time,door,lasted,price,power,sensor,cook
Word 1,popcorn,open,month,large,level,reheat,food
Word 2,button,latch,working,fit,power level,sensor reheat,heat
Word 3,cooking,close,warranty,size,watt,food,evenly
Word 4,setting,door latch,service,big,inverter,defrost,fast
Word 5,minute,push,repair,space,1250,function,quickly
Word 6,bag,button,died,counter,1250 watt,button,cook food
Word 7,used,door open,new,need,setting,reheat function,cook fast
Word 8,second,problem,model,kitchen,power setting,sensor cook,heat food
Word 9,long,hard,purchased,needed,watt power,used,powerful


In [ ]:
# df_featurizer(df2, topic_keywords2, topic_labels2, n_components=topics2, column_names_known = 'Y')

In [ ]:
# df_featurizer(df3, topic_keywords3, topic_labels3, n_components=topics3, column_names_known = 'Y')

In [64]:
def softmax(v, temperature=1.0):
    '''
    A heuristic to convert arbitrary positive values into probabilities.
    See: https://en.wikipedia.org/wiki/Softmax_function
    '''
    expv = np.exp(v / temperature)
    s = np.sum(expv)
    return expv / s

In [65]:
def analyze_reviews(W, topic_labels):
    '''
    Print an analysis of a single Amazon review, including the review ID
    and a summary of which topics it represents. The topics are identified
    via the hand-labels which were assigned by the user.
    '''
    topic_count = len(topic_labels)
    topic_percentages = [[] for i in range(0, topic_count)]

    for idx, i in enumerate(W):  
        probs = softmax(W[idx], temperature=0.01)
        topic_list_counter = 0
        for prob, label in zip(probs, topic_labels):
            topic_percentages[topic_list_counter].append(round(prob, 5))
            topic_list_counter += 1
        
    return pd.DataFrame(dict(zip(topic_labels, topic_percentages)))

In [103]:
top_df1 = analyze_reviews(W1, topic_labels1)

In [ ]:
# top_df2 = analyze_reviews(W2, topic_labels2)

In [ ]:
# top_df3 = analyze_reviews(W3, topic_labels3)

In [194]:
top_df1

,programmed features,door,broke,size,power,cooks evenly
0,0.00000,0.99988,0.00000,0.00000,0.00000,0.00000
1,0.00000,0.00000,0.00000,0.00000,0.00000,0.99999
2,0.00013,0.00013,0.00013,0.99912,0.00013,0.00021
3,0.01287,0.01287,0.91099,0.02409,0.01287,0.01287
4,0.00000,0.99998,0.00000,0.00000,0.00000,0.00001
...,...,...,...,...,...,...
1843,0.00856,0.00382,0.00382,0.67517,0.00426,0.10002
1844,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
1845,0.00005,0.00005,0.00005,0.00534,0.00005,0.98342
1846,0.00015,0.00015,0.00034,0.00015,0.00015,0.00016


In [68]:
top_df1.loc[0]

popcorn                  0.00010
door                     0.99988
short lifespan           0.00000
size                     0.00000
power                    0.00000
programmed functions     0.00000
cooks fast and evenly    0.00000
Name: 0, dtype: float64

In [74]:
def feature_score(top_df):    
    feature_score = {}
    for _ in range(len(top_df.columns)):
        key = top_df.columns[_]
        feature_score[key] = round(sum(top_df[key])/(len(top_df)),5)
    return feature_score

In [77]:
feature_score1 = feature_score(top_df1)
sorted(feature_score1.items(), key= lambda x: x[1], reverse=True)

[('size', 0.21387),
 ('short lifespan', 0.20912),
 ('door', 0.13818),
 ('cooks fast and evenly', 0.11958),
 ('popcorn', 0.11585),
 ('programmed functions', 0.10639),
 ('power', 0.09702)]

In [ ]:
# feature_score2 = feature_score(top_df2)
# sorted(feature_score2.items(), key= lambda x: x[1], reverse=True)

In [ ]:
# feature_score3 = feature_score(top_df3)
# sorted(feature_score3.items(), key= lambda x: x[1], reverse=True)

In [190]:
topic_words_df1 = nmf_featurizer(df1, topic_keywords1, topic_labels1, column_names_known ='Y')
# topic_words_df2 = nmf_featurizer(df2, topic_keywords2, topic_labels2, column_names_known ='Y')
# topic_words_df3 = nmf_featurizer(df3, topic_keywords3, topic_labels3, column_names_known ='Y')


In [191]:
topic_words_df1

topic_theme,programmed features,door,broke,size,power,programmed features,cooks evenly
Word 0,time,door,lasted,price,power,sensor,cook
Word 1,popcorn,open,month,large,level,reheat,food
Word 2,button,latch,working,fit,power level,sensor reheat,heat
Word 3,cooking,close,warranty,size,watt,food,evenly
Word 4,setting,door latch,service,big,inverter,defrost,fast
Word 5,minute,push,repair,space,1250,function,quickly
Word 6,bag,button,died,counter,1250 watt,button,cook food
Word 7,used,door open,new,need,setting,reheat function,cook fast
Word 8,second,problem,model,kitchen,power setting,sensor cook,heat food
Word 9,long,hard,purchased,needed,watt power,used,powerful


In [105]:
review_df_columns1 = (['review_body'] + topic_labels1)
# review_df_columns2 = (['review_body'] + topic_labels2)
# review_df_columns3 = (['review_body'] + topic_labels3)

In [106]:
review_df1 = pd.concat([df1, top_df1.reindex(df1.index)], axis=1)[review_df_columns1]
# review_df2 = pd.concat([df2, top_df2.reindex(df2.index)], axis=1)[review_df_columns2]
# review_df3 = pd.concat([df3, top_df3.reindex(df3.index)], axis=1)[review_df_columns3]

In [94]:
def review_checker(review_df, num_samples):
    review_df = review_df
    review_index = []
    for _ in range(num_samples):
        review_index.append(random.randint(0, len(review_df)))
        
    for _ in review_index:
        print('Review ' + str(_))
        print(' ')
        print(review_df.review_body[_])
        print(' ')
        print(review_df.loc[_][1:])
        print(' ')
        print('___________________________________________________')
        print(' ')
    

In [156]:
len(W1)

1848

In [219]:
W1[:,1].argsort()[-20:]

array([  14, 1018,  324, 1340,   31, 1113, 1540,  601,  333,  577,  706,
        554,  279, 1608,  211,  402,  133, 1426,   60, 1280])

In [189]:
H1[:,0].argsort()[:]

array([0, 4, 5, 6, 2, 1, 3])

In [221]:
df1.review_body[1018]

"Great Microwave.<br />It does what it supposed, but we are still learning the heating times and heat power levels because it's different then our old one."

In [269]:
np.array(vectorizer.get_feature_names()).take(nmf1.components_[6].argsort()[:])[:100]

array(["'0'", 'money', 'monday', 'moisture', 'modest', 'modern',
       'moderate', 'model wanted', 'model timer', 'model time',
       'model smaller', 'money built', 'model size', 'model problem',
       'model previous', 'model pretty', 'model number heard',
       'model number', 'model met expectation', 'model met',
       'model magnetron', 'model longer', 'model lasted', 'model similar',
       'money fit', 'money paid', "money's", 'month latch', 'month heavy',
       'month heating', 'month heat', 'month having', 'month gave',
       'month fine', 'month disappointed', 'month died', 'month decided',
       'month date', 'month cook', 'month complaint', 'month called',
       'month buyer beware', 'month buyer', 'month basic',
       'month ago started', 'month ago push', 'month', 'monster',
       'monitor', "money's worth", 'model knock', 'month later',
       'model inverter technology', 'model h765bf', 'minute time',
       'minute thirty second', 'minute thirty', 'minute se

In [161]:

for idx, i in enumerate(W1[:,2].argsort()[:]):
    if i == 368:
        print(idx)

123


'Shipped like the speed of light! Works great, very attractive in white, too. Yep, pretty good deal, too. Okey dokey.'

In [159]:
review_df1.loc[368][1:]

programmed features    0.00015
door                   0.96719
broke                  0.00015
size                   0.03197
power                  0.00015
programmed features    0.00015
cooks evenly           0.00015
Name: 368, dtype: object

In [107]:
review_checker(5)

Review 368
 
I had a Panasonic Genius as my first micro wave oven and it lasted over 20 years compared to a Convection Sharp that need to be<br />repaired by the 2nd year and 3 times after that.  I have only had it a few weeks, but I already love it.  Would recommend it to everyone.  I am also very impressed with Amazon.com.  They do a<br />tremendous job in getting out the orders and exactly when they say they will deliver.  I am very<br />impressed.
 
programmed features    0.00015
door                   0.96719
broke                  0.00015
size                   0.03197
power                  0.00015
programmed features    0.00015
cooks evenly           0.00015
Name: 368, dtype: object
 
___________________________________________________
 
Review 218
 
The microwave has gotten 100% approval from the people that are using it.
 
programmed features    0.00101
door                   0.00017
broke                    9e-05
size                   0.00076
power                  0.00011
